In [1]:
import os
import dotenv as env
from pydantic import BaseModel
from openai import OpenAI
import json
import pandas
import pprint as pp
env.load_dotenv('../keys.env')

True

In [2]:
class HeadlineClassification(BaseModel):
    title: str                          #Do not change the name of this field as OpenAI is dumb. If you keep it as inputs, then only it returns the original text sent to it.
                                        # Will have to carefully write wrappers to ensure that the outcome df to be used in the final output has the right headers. 
    classification: str
    explanation: str

In [3]:
class listClassifications(BaseModel):
    output: list[HeadlineClassification]

In [4]:
class OpenAIModels:
    __client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
    __model_name= 'gpt-4.1-nano-2025-04-14'
    __prompt_file_path = '../prompts/open_ai_prompts.json'
    
    def __getPromptFromFile(self, type):
        with open(self.__prompt_file_path, 'r') as file:
            data = file.read()
        parsed_data = json.loads(data)
        prompt = parsed_data.get(type)
        return prompt
    
    def classify_headlines(self, input) :
        prompt = self.__getPromptFromFile('headlines_classifier') 
        response = self.__client.responses.parse(
                    model=self.__model_name,
                    #service_tier='default',
                    temperature=1.0,
                    instructions=prompt, 
                    input=input.to_json(),
                    text_format=listClassifications
            
                )
        #Since the textformat here is a class wrapper of lists
        # So the return response must be handled to strip the 'output' dictionary to get to the inner lists
        return(response.output_text)
        

In [5]:
# Tester

#model = OpenAIModels()
#hl = ['Write a one-sentence bedtime story about a unicorn', 
#        'Hello my man xx!', 
#        'Sumeet Bagadia recommended buying these seven stock to buy',
#        'OpenAI is inconsistent!',
#        ]
#hl_series = pandas.Series(hl)
#response = model.classify_headlines(hl_series)
#pp.pprint(json.loads(response)['output'])